# A chain and a Y

In [1]:
import string
from fractions import Fraction
from math import comb
from itertools import combinations
import numpy as np
import svvamp

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


We study configurations of the following form: a chain of $a$ elements, and a Y shape defined as follows:
* A chain of $b$ elements,
* Two chains of respectively $c$ and $d$ elements just above.

In total, there are $a + b + c + d$ elements.

## Basic computations

In [2]:
def nb_linear_extensions(a, b, c, d):
    return comb(c + d, c) * comb(a + b + c + d, a)
nb_linear_extensions(10, 4, 5, 7)

4206894120

The following functions return lists of dimension a + b + c + d (each part from bottom node to top node).

In [3]:
def nb_ancestors(a, b, c, d):
    return np.concatenate((
        np.array(range(a, 0, -1), dtype=int),
        np.array(range(b, 0, -1), dtype=int) + c + d,
        np.array(range(c, 0, -1), dtype=int),
        np.array(range(d, 0, -1), dtype=int),
    ))
nb_ancestors(10, 4, 5, 7)

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 16, 15, 14, 13,  5,  4,  3,
        2,  1,  7,  6,  5,  4,  3,  2,  1])

In [4]:
def nb_descendants(a, b, c, d):
    return np.concatenate((
        np.array(range(1, a + 1), dtype=int),
        np.array(range(1, b + 1), dtype=int),
        np.array(range(1, c + 1), dtype=int) + b,
        np.array(range(1, d + 1), dtype=int) + b,
    ))
nb_descendants(10, 4, 5, 7)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  1,  2,  3,  4,  5,  6,  7,
        8,  9,  5,  6,  7,  8,  9, 10, 11])

In [5]:
def delta(a, b, c, d):
    return nb_descendants(a, b, c, d) - nb_ancestors(a, b, c, d)
delta(10, 4, 5, 7)

array([ -9,  -7,  -5,  -3,  -1,   1,   3,   5,   7,   9, -15, -13, -11,
        -9,   0,   2,   4,   6,   8,  -2,   0,   2,   4,   6,   8,  10])

In [6]:
def rho(a, b, c, d):
    nb_desc = nb_descendants(a, b, c, d)
    nb_anc = nb_ancestors(a, b, c, d)
    return np.array([Fraction(int(d), int(d + a)) for d, a in zip(nb_desc, nb_anc)])
print(rho(10, 4, 5, 7))

[Fraction(1, 11) Fraction(2, 11) Fraction(3, 11) Fraction(4, 11)
 Fraction(5, 11) Fraction(6, 11) Fraction(7, 11) Fraction(8, 11)
 Fraction(9, 11) Fraction(10, 11) Fraction(1, 17) Fraction(2, 17)
 Fraction(3, 17) Fraction(4, 17) Fraction(1, 2) Fraction(3, 5)
 Fraction(7, 10) Fraction(4, 5) Fraction(9, 10) Fraction(5, 12)
 Fraction(1, 2) Fraction(7, 12) Fraction(2, 3) Fraction(3, 4)
 Fraction(5, 6) Fraction(11, 12)]


## Average Height

In [7]:
def average_normalized_height_a(a, b, c, d):
    # In fact, it depends only on a
    return np.array([Fraction(i + 1, a + 1) for i in range(a)])
average_normalized_height_a(6, None, None, None)

array([Fraction(1, 7), Fraction(2, 7), Fraction(3, 7), Fraction(4, 7),
       Fraction(5, 7), Fraction(6, 7)], dtype=object)

In [8]:
def average_height_a(a, b, c, d):
    return average_normalized_height_a(a, b, c, d) * (a + b + c + d + 1)
average_height_a(6, 1, 1, 1)

array([Fraction(10, 7), Fraction(20, 7), Fraction(30, 7), Fraction(40, 7),
       Fraction(50, 7), Fraction(60, 7)], dtype=object)

In [9]:
def average_normalized_height_b(a, b, c, d):
    # In fact, it does not depend on a
    # It would be the same with a chain of c + d above b.
    return np.array([Fraction(i+1, b + c + d + 1) for i in range(b)])
average_normalized_height_b(None, 6, 2, 2)

array([Fraction(1, 11), Fraction(2, 11), Fraction(3, 11), Fraction(4, 11),
       Fraction(5, 11), Fraction(6, 11)], dtype=object)

In [10]:
def average_height_b(a, b, c, d):
    return average_normalized_height_b(a, b, c, d) * (a + b + c + d + 1)
average_height_b(0, 6, 2, 2)

array([Fraction(1, 1), Fraction(2, 1), Fraction(3, 1), Fraction(4, 1),
       Fraction(5, 1), Fraction(6, 1)], dtype=object)

In [11]:
def average_height_c(a, b, c, d):
    result = np.zeros(c, dtype=int)
    for k in range(c):
        # Elements of c are: 0 ... (k-1) k (k+1) .. (c-1)
        smaller_from_c = k
        greater_from_c = c - 1 - k
        for smaller_from_a in range(a + 1):
            greater_from_a = a - smaller_from_a
            for smaller_from_d in range(d + 1):
                greater_from_d = d - smaller_from_d
                height = smaller_from_a + b + smaller_from_c + smaller_from_d + 1
                n_lin_ext = (
                    nb_linear_extensions(smaller_from_a, b, smaller_from_c, smaller_from_d)
                    * nb_linear_extensions(greater_from_a, 0, greater_from_c, greater_from_d)
                )
                result[k] += n_lin_ext * height
    nb_lin_ext = nb_linear_extensions(a, b, c, d)
    return np.array([Fraction(r, nb_lin_ext) for r in result])
average_height_c(2, 0, 7, 0)

array([Fraction(5, 4), Fraction(5, 2), Fraction(15, 4), Fraction(5, 1),
       Fraction(25, 4), Fraction(15, 2), Fraction(35, 4)], dtype=object)

In [12]:
def average_normalized_height_c(a, b, c, d):
    return average_height_c(a, b, c, d) / (a + b + c + d + 1)
average_normalized_height_c(2, 0, 7, 0)

array([Fraction(1, 8), Fraction(1, 4), Fraction(3, 8), Fraction(1, 2),
       Fraction(5, 8), Fraction(3, 4), Fraction(7, 8)], dtype=object)

In [13]:
def average_height_d(a, b, c, d):
    return average_height_c(a, b, d, c)
average_height_d(2, 0, 0, 7)

array([Fraction(5, 4), Fraction(5, 2), Fraction(15, 4), Fraction(5, 1),
       Fraction(25, 4), Fraction(15, 2), Fraction(35, 4)], dtype=object)

In [14]:
def average_normalized_height_d(a, b, c, d):
    return average_normalized_height_c(a, b, d, c)
average_normalized_height_d(2, 0, 0, 7)

array([Fraction(1, 8), Fraction(1, 4), Fraction(3, 8), Fraction(1, 2),
       Fraction(5, 8), Fraction(3, 4), Fraction(7, 8)], dtype=object)

In [15]:
def average_height(a, b, c, d):
    return np.concatenate((
        average_height_a(a, b, c, d),
        average_height_b(a, b, c, d),
        average_height_c(a, b, c, d),
        average_height_d(a, b, c, d),
    ))
average_height(2, 1, 7, 3)

array([Fraction(14, 3), Fraction(28, 3), Fraction(7, 6),
       Fraction(133, 48), Fraction(35, 8), Fraction(287, 48),
       Fraction(91, 12), Fraction(147, 16), Fraction(259, 24),
       Fraction(595, 48), Fraction(35, 8), Fraction(91, 12),
       Fraction(259, 24)], dtype=object)

In [16]:
def average_normalized_height(a, b, c, d):
    return np.concatenate((
        average_normalized_height_a(a, b, c, d),
        average_normalized_height_b(a, b, c, d),
        average_normalized_height_c(a, b, c, d),
        average_normalized_height_d(a, b, c, d),
    ))
average_normalized_height(2, 1, 7, 3)

array([Fraction(1, 3), Fraction(2, 3), Fraction(1, 12), Fraction(19, 96),
       Fraction(5, 16), Fraction(41, 96), Fraction(13, 24),
       Fraction(21, 32), Fraction(37, 48), Fraction(85, 96),
       Fraction(5, 16), Fraction(13, 24), Fraction(37, 48)], dtype=object)

## Kemeny order

In [17]:
def linear_extensions(chain_1, chain_2):
    # Linear extensions for two independent chains (separate connected components)
    x = len(chain_1)
    y = len(chain_2)
    if x == 0 or y == 0:
        yield np.array(np.concatenate((chain_1, chain_2)), dtype=int)
    else:
        for places_x in combinations(range(x + y), x):
            extension = np.zeros(x + y, dtype=int)
            extension[np.array(places_x)] = chain_1
            mask_y = np.ones(x + y, dtype=bool)
            mask_y[np.array(places_x)] = False
            extension[mask_y] = chain_2
            yield extension

In [18]:
for linext in linear_extensions([1, 2, 3], [42, 52]):
    print(linext)

[ 1  2  3 42 52]
[ 1  2 42  3 52]
[ 1  2 42 52  3]
[ 1 42  2  3 52]
[ 1 42  2 52  3]
[ 1 42 52  2  3]
[42  1  2  3 52]
[42  1  2 52  3]
[42  1 52  2  3]
[42 52  1  2  3]


In [19]:
for linext in linear_extensions([1, 2, 3], []):
    print(linext)

[1 2 3]


In [20]:
def profile_linear_extensions(a, b, c, d):
    # The numpy profile consisting of all linear extensions
    profile = []
    elements_a = np.array(range(a), dtype=int)
    elements_b = np.array(range(a, a + b), dtype=int)
    elements_c = np.array(range(a + b, a + b + c), dtype=int)
    elements_d = np.array(range(a + b + c, a + b + c + d), dtype=int)
    for extension_c_d in linear_extensions(chain_1=elements_c, chain_2=elements_d):
        extension_b_c_d = np.concatenate((elements_b, extension_c_d))
        for extension in linear_extensions(chain_1=elements_a, chain_2=extension_b_c_d):
            profile.append(extension)
    return np.array(profile)

In [21]:
profile_np = profile_linear_extensions(0, 1, 2, 7)
profile_np

array([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
       [0, 1, 3, 2, 4, 5, 6, 7, 8, 9],
       [0, 1, 3, 4, 2, 5, 6, 7, 8, 9],
       [0, 1, 3, 4, 5, 2, 6, 7, 8, 9],
       [0, 1, 3, 4, 5, 6, 2, 7, 8, 9],
       [0, 1, 3, 4, 5, 6, 7, 2, 8, 9],
       [0, 1, 3, 4, 5, 6, 7, 8, 2, 9],
       [0, 1, 3, 4, 5, 6, 7, 8, 9, 2],
       [0, 3, 1, 2, 4, 5, 6, 7, 8, 9],
       [0, 3, 1, 4, 2, 5, 6, 7, 8, 9],
       [0, 3, 1, 4, 5, 2, 6, 7, 8, 9],
       [0, 3, 1, 4, 5, 6, 2, 7, 8, 9],
       [0, 3, 1, 4, 5, 6, 7, 2, 8, 9],
       [0, 3, 1, 4, 5, 6, 7, 8, 2, 9],
       [0, 3, 1, 4, 5, 6, 7, 8, 9, 2],
       [0, 3, 4, 1, 2, 5, 6, 7, 8, 9],
       [0, 3, 4, 1, 5, 2, 6, 7, 8, 9],
       [0, 3, 4, 1, 5, 6, 2, 7, 8, 9],
       [0, 3, 4, 1, 5, 6, 7, 2, 8, 9],
       [0, 3, 4, 1, 5, 6, 7, 8, 2, 9],
       [0, 3, 4, 1, 5, 6, 7, 8, 9, 2],
       [0, 3, 4, 5, 1, 2, 6, 7, 8, 9],
       [0, 3, 4, 5, 1, 6, 2, 7, 8, 9],
       [0, 3, 4, 5, 1, 6, 7, 2, 8, 9],
       [0, 3, 4, 5, 1, 6, 7, 8, 2, 9],
       [0, 3, 4, 5, 1, 6,

In [22]:
def kemeny_order(a, b, c, d):
    profile_np = profile_linear_extensions(a, b, c, d)
    profile = svvamp.Profile(preferences_rk=profile_np)
    election = svvamp.RuleKemeny()(profile)
    return election.candidates_by_scores_best_to_worst_

In [23]:
kemeny_order(0, 1, 2, 7)

array([0, 3, 4, 1, 5, 6, 7, 2, 8, 9])

In [24]:
def print_order(order):
    print("(" + "".join([string.ascii_lowercase[i] for i in order]) + ")")

In [25]:
print_order(kemeny_order(0, 1, 2, 7))

(adebfghcij)


## A simple example

In [26]:
a, b, c, d = 0, 1, 3, 2

In [27]:
delta(a, b, c, d)

array([-5, -1,  1,  3,  0,  2])

In [28]:
rho(a, b, c, d)

array([Fraction(1, 7), Fraction(2, 5), Fraction(3, 5), Fraction(4, 5),
       Fraction(1, 2), Fraction(3, 4)], dtype=object)

In [29]:
average_height(a, b, c, d)

array([Fraction(1, 1), Fraction(5, 2), Fraction(4, 1), Fraction(11, 2),
       Fraction(3, 1), Fraction(5, 1)], dtype=object)

In [30]:
print_order(kemeny_order(a, b, c, d))

(abecfd)


## Looking for a more complex example

In [31]:
def is_good_example(a, b, c, d):
    n = a + b + c + d
    my_rho = rho(a, b, c, d)
    if len(set(my_rho)) < n:  # Tie in rho
        return False
    my_delta = delta(a, b, c, d)
    if len(set(my_delta)) < n:  # Tie in delta
        return False
    order_rho = np.argsort(my_rho)
    order_delta = np.argsort(my_delta)
    if np.array_equal(order_rho, order_delta):  # rho and delta agree
        return False
    my_p = average_normalized_height(a, b, c, d)
    if len(set(my_p)) < n:  # Tie in p
        return False
    order_p = np.argsort(my_p)
    if np.array_equal(order_p, order_rho):  # p and rho agree
        return False
    if np.array_equal(order_p, order_delta):  # p and delta agree
        return False
    order_kemeny = kemeny_order(a, b, c, d)
    if np.array_equal(order_kemeny, order_rho):  # kemeny and rho agree
        return False
    if np.array_equal(order_kemeny, order_delta):  # kemeny and delta agree
        return False
    if np.array_equal(order_kemeny, order_p):  # kemeny and p agree
        return False
    return True

In [32]:
def find_minimal_example():
    n = 0
    results = []
    found = False
    while not found:
        n += 1
        for a in range(n + 1):
            for b in range(n - a + 1):
                remains = n - a - b
                for c in range(remains // 2 + 1):
                    d = n - a - b - c
                    print(f"Testing {a=}, {b=}, {c=}, {d=}...")
                    if is_good_example(a, b, c, d):
                        results.append((a, b, c, d))
                        found = True
    return results

In [33]:
results = find_minimal_example()

Testing a=0, b=0, c=0, d=1...
Testing a=0, b=1, c=0, d=0...
Testing a=1, b=0, c=0, d=0...
Testing a=0, b=0, c=0, d=2...
Testing a=0, b=0, c=1, d=1...
Testing a=0, b=1, c=0, d=1...
Testing a=0, b=2, c=0, d=0...
Testing a=1, b=0, c=0, d=1...
Testing a=1, b=1, c=0, d=0...
Testing a=2, b=0, c=0, d=0...
Testing a=0, b=0, c=0, d=3...
Testing a=0, b=0, c=1, d=2...
Testing a=0, b=1, c=0, d=2...
Testing a=0, b=1, c=1, d=1...
Testing a=0, b=2, c=0, d=1...
Testing a=0, b=3, c=0, d=0...
Testing a=1, b=0, c=0, d=2...
Testing a=1, b=0, c=1, d=1...
Testing a=1, b=1, c=0, d=1...
Testing a=1, b=2, c=0, d=0...
Testing a=2, b=0, c=0, d=1...
Testing a=2, b=1, c=0, d=0...
Testing a=3, b=0, c=0, d=0...
Testing a=0, b=0, c=0, d=4...
Testing a=0, b=0, c=1, d=3...
Testing a=0, b=0, c=2, d=2...
Testing a=0, b=1, c=0, d=3...
Testing a=0, b=1, c=1, d=2...
Testing a=0, b=2, c=0, d=2...
Testing a=0, b=2, c=1, d=1...
Testing a=0, b=3, c=0, d=1...
Testing a=0, b=4, c=0, d=0...
Testing a=1, b=0, c=0, d=3...
Testing a=

Testing a=3, b=3, c=1, d=2...
Testing a=3, b=4, c=0, d=2...
Testing a=3, b=4, c=1, d=1...
Testing a=3, b=5, c=0, d=1...
Testing a=3, b=6, c=0, d=0...
Testing a=4, b=0, c=0, d=5...
Testing a=4, b=0, c=1, d=4...
Testing a=4, b=0, c=2, d=3...
Testing a=4, b=1, c=0, d=4...
Testing a=4, b=1, c=1, d=3...
Testing a=4, b=1, c=2, d=2...
Testing a=4, b=2, c=0, d=3...
Testing a=4, b=2, c=1, d=2...
Testing a=4, b=3, c=0, d=2...
Testing a=4, b=3, c=1, d=1...
Testing a=4, b=4, c=0, d=1...
Testing a=4, b=5, c=0, d=0...
Testing a=5, b=0, c=0, d=4...
Testing a=5, b=0, c=1, d=3...
Testing a=5, b=0, c=2, d=2...
Testing a=5, b=1, c=0, d=3...
Testing a=5, b=1, c=1, d=2...
Testing a=5, b=2, c=0, d=2...
Testing a=5, b=2, c=1, d=1...
Testing a=5, b=3, c=0, d=1...
Testing a=5, b=4, c=0, d=0...
Testing a=6, b=0, c=0, d=3...
Testing a=6, b=0, c=1, d=2...
Testing a=6, b=1, c=0, d=2...
Testing a=6, b=1, c=1, d=1...
Testing a=6, b=2, c=0, d=1...
Testing a=6, b=3, c=0, d=0...
Testing a=7, b=0, c=0, d=2...
Testing a=

Testing a=0, b=5, c=2, d=5...
Testing a=0, b=5, c=3, d=4...
Testing a=0, b=6, c=0, d=6...
Testing a=0, b=6, c=1, d=5...
Testing a=0, b=6, c=2, d=4...
Testing a=0, b=6, c=3, d=3...
Testing a=0, b=7, c=0, d=5...
Testing a=0, b=7, c=1, d=4...
Testing a=0, b=7, c=2, d=3...
Testing a=0, b=8, c=0, d=4...
Testing a=0, b=8, c=1, d=3...
Testing a=0, b=8, c=2, d=2...
Testing a=0, b=9, c=0, d=3...
Testing a=0, b=9, c=1, d=2...
Testing a=0, b=10, c=0, d=2...
Testing a=0, b=10, c=1, d=1...
Testing a=0, b=11, c=0, d=1...
Testing a=0, b=12, c=0, d=0...
Testing a=1, b=0, c=0, d=11...
Testing a=1, b=0, c=1, d=10...
Testing a=1, b=0, c=2, d=9...
Testing a=1, b=0, c=3, d=8...
Testing a=1, b=0, c=4, d=7...
Testing a=1, b=0, c=5, d=6...
Testing a=1, b=1, c=0, d=10...
Testing a=1, b=1, c=1, d=9...
Testing a=1, b=1, c=2, d=8...
Testing a=1, b=1, c=3, d=7...
Testing a=1, b=1, c=4, d=6...
Testing a=1, b=1, c=5, d=5...
Testing a=1, b=2, c=0, d=9...
Testing a=1, b=2, c=1, d=8...
Testing a=1, b=2, c=2, d=7...
Tes

Testing a=0, b=4, c=2, d=8...
Testing a=0, b=4, c=3, d=7...
Testing a=0, b=4, c=4, d=6...
Testing a=0, b=4, c=5, d=5...
Testing a=0, b=5, c=0, d=9...
Testing a=0, b=5, c=1, d=8...
Testing a=0, b=5, c=2, d=7...
Testing a=0, b=5, c=3, d=6...
Testing a=0, b=5, c=4, d=5...
Testing a=0, b=6, c=0, d=8...
Testing a=0, b=6, c=1, d=7...
Testing a=0, b=6, c=2, d=6...
Testing a=0, b=6, c=3, d=5...
Testing a=0, b=6, c=4, d=4...
Testing a=0, b=7, c=0, d=7...
Testing a=0, b=7, c=1, d=6...
Testing a=0, b=7, c=2, d=5...
Testing a=0, b=7, c=3, d=4...
Testing a=0, b=8, c=0, d=6...
Testing a=0, b=8, c=1, d=5...
Testing a=0, b=8, c=2, d=4...
Testing a=0, b=8, c=3, d=3...
Testing a=0, b=9, c=0, d=5...
Testing a=0, b=9, c=1, d=4...
Testing a=0, b=9, c=2, d=3...
Testing a=0, b=10, c=0, d=4...
Testing a=0, b=10, c=1, d=3...
Testing a=0, b=10, c=2, d=2...
Testing a=0, b=11, c=0, d=3...
Testing a=0, b=11, c=1, d=2...
Testing a=0, b=12, c=0, d=2...
Testing a=0, b=12, c=1, d=1...
Testing a=0, b=13, c=0, d=1...
Te

Testing a=0, b=6, c=4, d=5...
Testing a=0, b=7, c=0, d=8...
Testing a=0, b=7, c=1, d=7...
Testing a=0, b=7, c=2, d=6...
Testing a=0, b=7, c=3, d=5...
Testing a=0, b=7, c=4, d=4...
Testing a=0, b=8, c=0, d=7...
Testing a=0, b=8, c=1, d=6...
Testing a=0, b=8, c=2, d=5...
Testing a=0, b=8, c=3, d=4...
Testing a=0, b=9, c=0, d=6...
Testing a=0, b=9, c=1, d=5...
Testing a=0, b=9, c=2, d=4...
Testing a=0, b=9, c=3, d=3...
Testing a=0, b=10, c=0, d=5...
Testing a=0, b=10, c=1, d=4...
Testing a=0, b=10, c=2, d=3...
Testing a=0, b=11, c=0, d=4...
Testing a=0, b=11, c=1, d=3...
Testing a=0, b=11, c=2, d=2...
Testing a=0, b=12, c=0, d=3...
Testing a=0, b=12, c=1, d=2...
Testing a=0, b=13, c=0, d=2...
Testing a=0, b=13, c=1, d=1...
Testing a=0, b=14, c=0, d=1...
Testing a=0, b=15, c=0, d=0...
Testing a=1, b=0, c=0, d=14...
Testing a=1, b=0, c=1, d=13...
Testing a=1, b=0, c=2, d=12...
Testing a=1, b=0, c=3, d=11...
Testing a=1, b=0, c=4, d=10...
Testing a=1, b=0, c=5, d=9...
Testing a=1, b=0, c=6, 

Testing a=13, b=0, c=1, d=1...
Testing a=13, b=1, c=0, d=1...
Testing a=13, b=2, c=0, d=0...
Testing a=14, b=0, c=0, d=1...
Testing a=14, b=1, c=0, d=0...
Testing a=15, b=0, c=0, d=0...


In [34]:
results

[(3, 7, 2, 3)]

In [35]:
a, b, c, d = results[0]
a, b, c, d

(3, 7, 2, 3)

In [36]:
profile = svvamp.Profile(preferences_rk=profile_linear_extensions(a, b, c, d))
n_extensions = profile.n_v
def kemeny_score(order):
    return np.sum(np.tril(profile.matrix_duels_rk[order, :][:, order], -1)) / n_extensions

In [37]:
d_criterion_order = {
    'delta': np.argsort(delta(a, b, c, d)),
    'rho': np.argsort(rho(a, b, c, d)),
    'p': np.argsort(average_normalized_height(a, b, c, d)),
    'kemeny': kemeny_order(a, b, c, d), 
}
for criterion, order in d_criterion_order.items():
    print(criterion)
    print_order(order)
    print(kemeny_score(order))
    print()

delta
(defghaibjcmknlo)
10.03076923076923

rho
(defaghibjmcknlo)
8.914285714285715

p
(defaghibjmkcnlo)
8.661978021978022

kemeny
(deafghibjmknclo)
8.61054945054945



In [38]:
average_normalized_height(a, b, c, d)

array([Fraction(1, 4), Fraction(1, 2), Fraction(3, 4), Fraction(1, 13),
       Fraction(2, 13), Fraction(3, 13), Fraction(4, 13), Fraction(5, 13),
       Fraction(6, 13), Fraction(7, 13), Fraction(9, 13),
       Fraction(11, 13), Fraction(17, 26), Fraction(10, 13),
       Fraction(23, 26)], dtype=object)

In [39]:
heights = [Fraction(int(num), 4550 * (a + b + c + d + 1)) for num in average_normalized_height(a, b, c, d) * (a + b + c + d + 1) * 4550]
heights

[Fraction(1, 4),
 Fraction(1, 2),
 Fraction(3, 4),
 Fraction(1, 13),
 Fraction(2, 13),
 Fraction(3, 13),
 Fraction(4, 13),
 Fraction(5, 13),
 Fraction(6, 13),
 Fraction(7, 13),
 Fraction(9, 13),
 Fraction(11, 13),
 Fraction(17, 26),
 Fraction(10, 13),
 Fraction(23, 26)]